# Data ETL

In [3]:
import pandas as pd

## Historical

In [5]:
# get historical data 
historical_path = "historical_iah.py"

def historical():
    %run $historical_path
    return historical_iah

aih_hist = historical()

In [70]:
# Drop fully empty columns from aih_hist DataFrame
aih_hist.dropna(axis='columns', how='all', inplace=True)

In [50]:
# group by carrier code 
carrier_group = aih_hist.groupby(['Carrier Code']).nunique().reset_index()

In [56]:
carrier_group

,Carrier Code,Date (MM/DD/YYYY),Flight Number,Tail Number,Destination Airport,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Wheels-off time,Taxi-Out time (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes)
0,9E,1137,205,220,7,164,725,69,152,214,762,68,87,22,67,2,142
1,AA,3772,721,1633,10,693,1153,132,250,596,1160,138,421,208,185,22,410
2,AS,3770,26,194,2,61,552,38,143,189,570,69,99,50,80,11,107
3,B6,1078,6,224,3,102,562,42,116,217,582,58,91,13,83,1,137
4,DL,3772,760,828,8,605,1102,119,206,534,1113,112,409,105,175,11,288
5,EV,2830,1693,437,98,971,1193,182,261,599,1194,151,442,313,308,8,325
6,F9,3720,91,175,14,569,1265,105,224,345,1270,91,165,40,155,1,247
7,MQ,2406,139,202,5,198,923,61,207,277,941,89,158,72,139,5,178
8,NK,3029,213,217,24,662,1258,161,272,448,1258,115,293,172,291,40,284
9,OO,3772,1718,538,114,959,1209,213,287,666,1209,143,543,287,216,66,352


In [51]:
dest_airport = aih_hist.groupby(['Destination Airport']).nunique().reset_index()

In [65]:
df_dest_airport = pd.DataFrame(dest_airport, columns=['Destination Airport'])

In [66]:
df_carrier_key = pd.DataFrame(carrier_group,columns=['Carrier Code'])

## Current

In [8]:
# get historical data 
current_path = "current_iah.py"

def current():
    %run $current_path
    return current_iah

aih_curr = current()

In [72]:
# Drop the last and second-to-last columns from aih_hist DataFrame
aih_curr.drop(aih_curr.columns[-2:], axis=1, inplace=True)

C:\Users\MOTHERSHIP\AppData\Local\Temp\ipykernel_7184\1296296145.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  aih_curr.drop(aih_curr.columns[-2:], axis=1, inplace=True)


In [63]:
# group by carrier code 
carriers_aih = aih_curr.groupby(['OP_UNIQUE_CARRIER']).nunique()

In [79]:
carriers_aih_flight = aih_curr.groupby(['OP_UNIQUE_CARRIER','OP_CARRIER_FL_NUM']).nunique()

In [80]:
carriers_aih_flight.drop(aih_curr.columns[-2:], axis=1, inplace=True)

### DB Load

In [67]:
db_df_export = {
    'raw_current': aih_curr,
    'raw_past' : aih_hist,
    'airport_key': df_dest_airport,
    'carrier_key': df_carrier_key,
    'carrier_flnum':carriers_aih_flight
}

In [58]:
import sqlite3

In [59]:
conn = sqlite3.connect('rawdata.db')

In [81]:
# Iterate over the dictionary and load DataFrames into SQLite with error handling
for table_name, df in db_df_export.items():
    try:
        # Use the table_name as the table name in the database
        df.to_sql(table_name, conn, if_exists='replace', index=False)
    except Exception as e:
        print(f"Error occurred while loading DataFrame '{table_name}': {e}")


Error occurred while loading DataFrame 'carrier_flnum': duplicate column name: None


In [ ]:
import os
folderpath = "../rawdata/clean"

# Export DataFrames in db_df_export dictionary to CSV and JSON files
for table_name, df in db_df_export.items():
    # Export to CSV
    csv_filename = os.path.join(folderpath,f"{table_name}.csv")
    df.to_csv(csv_filename, index=False)
    print(f"DataFrame '{table_name}' exported to CSV: {csv_filename}")

    # Export to JSON
    json_filename = os.path.join(folderpath,f"{table_name}.json")
    df.to_json(json_filename, orient='records')
    print(f"DataFrame '{table_name}' exported to JSON: {json_filename}")
